In [ ]:
import os
from  PIL import Image
import numpy as np
import cv2
face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt2.xml')

In [ ]:
base_dir = os.path.dirname(os.path.abspath('__file__'))
base_dir

In [ ]:
image_dir = os.path.join(base_dir,"images")

In [ ]:
image_dir

In [ ]:
x_train = []
y_labels = []
current_id =0
label_ids = {}


In [ ]:
for root , dirs , files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("PNG") or file.endswith("jpeg") or file.endswith("JPEG") or file.endswith("JPG"):
            path = os.path.join(root,file)
            
            #print(path)
            
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            ids = label_ids[label]
            
            
            gray = Image.open(path).convert("L") #convert into grayscale
                                  
            image_array = np.array(gray) #take pixel value of image in numpy array
            
#             resized_gray = cv2.resize(image_array ,(200,200))

            faces = face_cascade.detectMultiScale(image_array,scaleFactor=1.5,minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h , x:x+w]
                roi = cv2.resize(roi,(100,100))     
                x_train.append(roi)                    
                y_labels.append(ids)
                
import pickle

with open('labels.pickle','wb') as f:
    pickle.dump(label_ids, f)

In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = np.array(x_train)
y_labels = np.array(y_labels)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[3] , cmap ='gist_gray')

In [ ]:
def one_hot_encode(vector , possible_values):
    '''
    For use to one-hot encode the 10- possible labels
    '''
    n = len(vector)
    out = np.zeros((n, possible_values))
    out[range(n), vector] = 1
    return out

In [ ]:
class ImageDataHelper():
    
    def __init__(self):
        
        self.i = 0
        self.training_images = None
        self.training_labels = None
        
        self.test_images = None
        self.test_labels = None
        
        
    def set_up_images(self):
                
        self.training_images = X
        train_len = len(self.training_images)
        #print(train_len)
        
        self.training_images = self.training_images.reshape(train_len ,1,100,100).transpose(0,2,3,1)/255
        #print(self.training_images[2].shape)
        #print(self.training_images.max())
        
        self.training_labels = one_hot_encode(y_labels, len(np.unique(y_labels)))
        #print(self.training_labels)
        
        self.test_images = X
        test_len = len(self.test_images)        
        self.test_images = self.test_images.reshape(test_len ,1,100,100).transpose(0,2,3,1)/255
        
        self.test_labels = one_hot_encode(y_labels, len(np.unique(y_labels)))
        
        

    def next_batch(self, batch_size):
        x = self.training_images[self.i:self.i+batch_size].reshape(1,100,100,1)
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_images)
        return x, y
        

In [ ]:
b = ImageDataHelper()
b.set_up_images()

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,100,100,1], name="inputx")
y_true = tf.placeholder(tf.float32,shape=[None,len(np.unique(y_labels))],name="inputy")

In [ ]:
hold_prob = tf.placeholder(tf.float32 , name="hold_prob")

In [ ]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist , name = "weights")

In [ ]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals , name="bias")

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [ ]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [ ]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [ ]:
convo_1 = convolutional_layer(x,shape=[4,4,1,100])
convo_1_pooling = max_pool_2by2(convo_1)

In [ ]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[4,4,100,200])
convo_2_pooling = max_pool_2by2(convo_2)
convo_2_pooling.shape

In [ ]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,25*25*200])
convo_2_flat.shape

In [ ]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [ ]:
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [ ]:
y_pred = normal_full_layer(full_one_dropout,len(np.unique(y_labels)))
y_pred = tf.identity(y_pred , name="y_pred")

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
        sess.run(init)

        for i in range(5000):

            batch = b.next_batch(1)


            sess.run(train, feed_dict={x:batch[0], y_true: batch[1], hold_prob: 0.5})

            if i%100 == 0:
                
                print(y_pred)
                print('Currently on step {}'.format(i))
                print('Accuracy is:')
                # Test the Train Model
                matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

                acc = tf.reduce_mean(tf.cast(matches,tf.float32))

                print(sess.run(acc,feed_dict={x:b.test_images,y_true:b.test_labels,hold_prob:1.0}))
                print('\n')


        saver.save(sess,'save_model/face_model')